<a href="https://colab.research.google.com/github/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install mmcv
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 21799, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21799 (delta 6), reused 9 (delta 3), pack-reused 21781
Receiving objects: 100% (21799/21799), 25.21 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (15311/15311), done.
/home/ueidarchen/Homework 2/mmdetection
Obtaining file:///home/ueidarchen/Homework%202/mmdetection
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmdet


In [2]:
# Download the training data
!gdown https://drive.google.com/u/1/uc?id=1lrKueI4HrySQDGvpkilQN9BfaMUN7hZi
!unzip -q train.zip

Downloading...
From: https://drive.google.com/u/1/uc?id=1lrKueI4HrySQDGvpkilQN9BfaMUN7hZi
To: /home/ueidarchen/Homework 2/mmdetection/train.zip
100%|█████████████████████████████████████████| 420M/420M [00:01<00:00, 266MB/s]


In [3]:
# Clone project
!rm -rf VRDL
!git clone https://github.com/udchen19/VRDL.git
!pip install -r VRDL/HW2/requirements.txt
!cp VRDL/HW2/config.py .

Cloning into 'VRDL'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 32 (delta 5), reused 28 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [4]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu102 True
2.18.1
10.2
GCC 7.3


## Perform inference with a MMDet detector
MMDetection already provides high level APIs to do inference and training.

In [5]:
# Mat file preprocessing
import h5py, json
import numpy as np
from PIL import Image

DATA_DIR = 'train'

mat = h5py.File(f'{DATA_DIR}/digitStruct.mat', 'r')
bboxes = mat['digitStruct/bbox']
names = mat['digitStruct/name']

In [6]:
# Get dict list from mat file

KEYS = ['label', 'left', 'top', 'width', 'height']
ann_list = []

for bbox, name in zip(bboxes, names):
    filename = ''.join(chr(char[0]) for char in mat[name[0]][()])
    img = Image.open(f'{DATA_DIR}/{filename}')
    width, height = img.size
    if mat[bbox[0]][KEYS[0]].shape[0] > 1:
        attr = np.array([[mat[mat[bbox[0]][key][k][0]][0][0]
                          for k in range(mat[bbox[0]][key].shape[0])]
                             for key in KEYS]).T.astype(int)
    else:
        attr = np.array([mat[bbox[0]][key][0] for key in KEYS]).T.astype(int)
    labels = (attr[:, 0] % 10).tolist()
    bboxes_data = attr[:, 1:].tolist()
    obj = dict(
        filename = filename,
        width = width,
        height = height,
        ann = dict(
            bboxes = bboxes_data,
            labels = labels
        )
    )
    ann_list.append(obj)

In [7]:
# Export dict list to JSON

import random, json

DATA_SIZE = len(ann_list)
TRAIN_RATIO = 0.7
TRAIN_SIZE = int(DATA_SIZE * TRAIN_RATIO)
VAL_SIZE = DATA_SIZE - TRAIN_SIZE

img_all = [ann['filename'] for ann in ann_list]
random.shuffle(img_all)
img_train = set(img_all[: TRAIN_SIZE])
img_val = set(img_all[TRAIN_SIZE: ])
ann_train = [ann for ann in ann_list if ann['filename'] in img_train]
ann_val = [ann for ann in ann_list if ann['filename'] in img_val]

with open(f'{DATA_DIR}/ann_train.json', 'w') as f:
    f.write(json.dumps(ann_train))
with open(f'{DATA_DIR}/ann_val.json', 'w') as f:
    f.write(json.dumps(ann_val))

In [8]:
# Define and register custom dataset

import os.path as osp
import copy

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module()
class DigitDataset(CustomDataset):
    CLASSES = tuple(str(i) for i in range(10))
    def load_annotations(self, ann_file):
        ann_list = mmcv.load(self.ann_file)
        for i, a in enumerate(ann_list):
            ann_list[i]['ann']['bboxes'] = np.array(
                ann_list[i]['ann']['bboxes'], dtype=np.float32)
            # (x, y, w, h) -> #(xmin, ymin, xmax, ymax)
            ann_list[i]['ann']['bboxes'][:, 2:] += ann_list[i]['ann']['bboxes'][:, :2] - 1
            ann_list[i]['ann']['labels'] = np.array(
                ann_list[i]['ann']['labels'], dtype=np.long)
        return ann_list

In [9]:
# Download Pretrained weights
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth -O checkpoints/faster_rcnn.pth

--2021-11-25 16:31:18--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.5
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn.pth’

checkpoints/faster_ 100%[===================>] 159.54M  68.3MB/s    in 2.3s    

2021-11-25 16:31:20 (68.3 MB/s) - ‘checkpoints/faster_rcnn.pth’ saved [167287506/167287506]



In [10]:
# Import Configuration File
from mmcv import Config
cfg = Config.fromfile('config.py')
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Train detector
train_detector(model, datasets, cfg, distributed=False, validate=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/ueidarchen/Homework 2/mmdetection/mmdet/datasets/custom.py:157: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')
2021-11-25 16:31:25,359 - mmdet - INFO - load checkpoint from local path: checkpoints